In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI Sales Cross Selling and UpSelling\data\dataset\Office & Administration Series.csv")
df.head()

,Unnamed: 0,company_ID,company_name,company_pic,company_address,company_phone,company_email,company_field,transaction_date,level_training_process,...,interest_in_discount_program,training_ID,trainer_name,training_total_meetings,training_price,discount_training,training_type,training_name,cross_selling,upselling
0,0,P0001,Tribakti Inspektama,Lasmanto Pratiwi,"Jalan Cihampelas No. 282\nPontianak, Kalimanta...",+62 (183) 655 1415,muhammadnasyidah@example.org,Aktivitas Keuangan dan Asuransi,2020-01-01,2,...,2,B0001,Alam,8,3000000,0,Office & Administration Series,Interactive Dashboard with Microsoft Excel,Managing General Affairs Excellence,Electronic Filing Management With Simple Datab...
1,1,P0002,Bank Bukopin Syariah,Puti Gina Napitupulu,"Jalan Tubagus Ismail No. 02\nSukabumi, BB 74318",(097) 773 7113,januarkamaria@example.net,Aktivitas Badan Internasional Dan Badan Ekstra...,2020-01-02,3,...,2,B0002,Reza,11,4000000,10,Office & Administration Series,Excel & Powerpoint For Powerful Business Appli...,Excel & Powerpoint For Powerful Business Appli...,Managing General Affairs Excellence
2,2,P0003,Bank Bukopin Syariah,"Karna Purwanti, S.I.Kom","Gg. Pelajar Pejuang No. 660\nJambi, Jambi 60449",+62 (0801) 007-5669,rprasetyo@example.net,"Pengadaan Listrik, Gas, Uap/Air Panas Dan Udar...",2020-01-03,4,...,4,B0003,Haris,14,11000000,25,Office & Administration Series,Managing General Affairs Excellence,Electronic Filing Management With Simple Datab...,Powerful Database Analysis & Dashboard Reporti...
3,3,P0004,Kementrian Agama,Tania Yolanda,"Jl. Cihampelas No. 8\nTual, ST 59647",+62-0247-253-2150,iswahyudiraisa@example.net,Konstruksi,2020-01-04,2,...,3,B0004,Reza,11,11000000,10,Office & Administration Series,Excel & Powerpoint For Powerful Business Appli...,Excel & Powerpoint For Powerful Business Appli...,Managing General Affairs Excellence
4,4,P0005,M3kom,Zaenab Anggriawan,"Gang Moch. Ramdan No. 083\nPayakumbuh, SS 13214",(025) 406 2520,mulyono67@example.com,Informasi Dan Komunikasi,2020-01-05,5,...,5,B0005,Haris,17,11000000,50,Office & Administration Series,Powerful Database Analysis & Dashboard Reporti...,Secretary Development Program,Excel & Powerpoint For Powerful Business Appli...


In [3]:
df.drop([
    "Unnamed: 0", "company_ID", "company_name", "company_pic", "company_address", "company_phone", "company_email",
    "transaction_date", "training_ID", "possibility_of_purchasing_the_same_training", "cross_selling"
], axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   company_field                 5000 non-null   object 
 1   level_training_process        5000 non-null   int64  
 2   purchasing_time_patterns      5000 non-null   object 
 3   number_of_participants        5000 non-null   int64  
 4   group                         5000 non-null   object 
 5   total_payment                 5000 non-null   float64
 6   satisfaction                  5000 non-null   int64  
 7   communication_channel         5000 non-null   object 
 8   complaints                    5000 non-null   object 
 9   communication_quality         5000 non-null   object 
 10  frequency_engagement          5000 non-null   int64  
 11  brand_sustainability          5000 non-null   int64  
 12  rating_training               5000 non-null   int64  
 13  tre

In [4]:
columns = {}
for i in df.columns:
    if df[i].dtype == "category" or df[i].dtype == "object" :
        columns[i] = LabelEncoder().fit(df[i])
        df[i] = columns[i].transform(df[i])
             
scaler = RobustScaler().fit(df[[i for i in df.columns if i not in columns]])
df2 = pd.DataFrame(scaler.transform(df[[i for i in df.columns if i not in columns]]), columns=df[[i for i in df.columns if i not in columns]].columns)
for i in columns:
    df2[i] = df[i]
df2.head(5)

,level_training_process,number_of_participants,total_payment,satisfaction,frequency_engagement,brand_sustainability,rating_training,interest_in_discount_program,training_total_meetings,training_price,...,purchasing_time_patterns,group,communication_channel,complaints,communication_quality,tren_mode_factor,trainer_name,training_type,training_name,upselling
0,-1.00,-0.73,-0.79,-1.50,-1.00,-1.14,-1.50,-1.00,-1.00,-0.83,...,0,1,0,0,1,0,0,0,3,1
1,-0.50,-0.45,-0.56,-0.50,-0.73,-0.57,-0.50,-1.00,-0.50,-0.67,...,0,1,1,1,2,0,6,0,2,4
2,0.00,-0.09,0.53,0.00,-0.09,-0.29,0.00,0.00,0.00,0.50,...,1,0,2,2,0,1,2,0,4,6
3,-1.00,-0.45,0.05,-0.50,-0.45,-0.43,-1.00,-0.50,-0.50,0.50,...,2,1,2,1,2,0,6,0,2,4
4,0.50,0.55,0.80,0.50,0.27,0.43,0.50,0.50,0.50,0.50,...,1,0,2,2,0,1,2,0,6,2


In [5]:
X = df2.drop("upselling", axis=1)
y = df2["upselling"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyClassifier(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [00:22<00:00,  2.13it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 400
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 20
[LightGBM] [Info] Start training from score -2.273026
[LightGBM] [Info] Start training from score -2.021173
[LightGBM] [Info] Start training from score -1.476219
[LightGBM] [Info] Start training from score -4.216512
[LightGBM] [Info] Start training from score -1.184170
[LightGBM] [Info] Start training from score -4.183176
[LightGBM] [Info] Start training from score -1.684700
[LightGBM] [Info] Start training from score -4.233607


100%|██████████| 29/29 [00:23<00:00,  1.22it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.88,0.58,None,0.86,1.02
LGBMClassifier,0.88,0.58,None,0.86,1.39
ExtraTreesClassifier,0.88,0.57,None,0.86,0.75
RandomForestClassifier,0.88,0.57,None,0.85,0.85
AdaBoostClassifier,0.86,0.56,None,0.83,0.84
BaggingClassifier,0.85,0.55,None,0.83,0.52
SVC,0.81,0.53,None,0.79,1.22
KNeighborsClassifier,0.81,0.52,None,0.78,0.30
LinearDiscriminantAnalysis,0.79,0.51,None,0.77,0.06
